In [18]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import geojson
import seaborn as sns
from shapely.geometry import shape, GeometryCollection
from shapely.geometry import Polygon, MultiPolygon

import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDRegressor, SGDClassifier
from  math import radians





In [2]:
np.random.seed(0)

In [19]:
initial = pd.read_csv('D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/nyc_10%.csv' )

### MODIFYING THE FILE .CHANGE UNIT OF MEASUREMENT
(this step maynot be needed since the original file is parquet and it does not have PUWeekday, Unnamed, etc yet)

In [5]:
initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2383893 entries, 0 to 2383892
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   Unnamed: 0.1           int64  
 2   PUlon                  float64
 3   PUlat                  float64
 4   DOlon                  float64
 5   DOlat                  float64
 6   PUmonth                int64  
 7   PUhour                 int64  
 8   DOmonth                int64  
 9   DOhour                 int64  
 10  PUweekday              object 
 11  DOweekday              object 
 12  PUweekend              int64  
 13  DOweekend              int64  
 14  duration               int64  
 15  passenger_count        int64  
 16  trip_distance          float64
 17  RatecodeID             int64  
 18  PULocationID           int64  
 19  DOLocationID           int64  
 20  payment_type           int64  
 21  fare_amount            int64  
 22  extra             

In [6]:
initial.describe()

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,...,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06,2.383893e+06
mean,1.192035e+07,2.049954e+06,-7.397130e+01,4.075523e+01,-7.397209e+01,4.075674e+01,4.387643e+00,1.390791e+01,4.387851e+00,1.397905e+01,...,1.620915e+02,1.280152e+00,1.228197e+03,1.074228e+02,4.971996e+01,2.094094e+02,2.691236e+01,2.974743e-01,1.805115e+03,2.271044e+00
std,6.881517e+06,1.819938e+06,3.855052e-02,2.891074e-02,3.371494e-02,3.183477e-02,3.829750e+00,5.554415e+00,3.829617e+00,5.621469e+00,...,7.065835e+01,4.839824e-01,2.583023e+04,1.239919e+02,4.236119e+00,2.602302e+02,1.748163e+02,3.806790e-02,2.584107e+04,7.475942e-01
min,1.000000e+00,1.000000e+00,-7.423000e+01,4.053000e+01,-7.423000e+01,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.000000e-01,0.000000e+00,-2.500000e+00
25%,5.955808e+06,5.506750e+05,-7.399000e+01,4.074000e+01,-7.399000e+01,4.074000e+01,1.000000e+00,1.000000e+01,1.000000e+00,1.000000e+01,...,1.070000e+02,1.000000e+00,6.500000e+02,0.000000e+00,5.000000e+01,0.000000e+00,0.000000e+00,3.000000e-01,1.115000e+03,2.500000e+00
50%,1.192223e+07,1.330726e+06,-7.398000e+01,4.076000e+01,-7.398000e+01,4.076000e+01,2.000000e+00,1.400000e+01,2.000000e+00,1.500000e+01,...,1.620000e+02,1.000000e+00,9.000000e+02,5.000000e+01,5.000000e+01,1.860000e+02,0.000000e+00,3.000000e-01,1.416000e+03,2.500000e+00
75%,1.788336e+07,3.312270e+06,-7.397000e+01,4.077000e+01,-7.396000e+01,4.078000e+01,8.000000e+00,1.800000e+01,8.000000e+00,1.800000e+01,...,2.340000e+02,2.000000e+00,1.350000e+03,2.500000e+02,5.000000e+01,2.850000e+02,0.000000e+00,3.000000e-01,1.930000e+03,2.500000e+00
max,2.383893e+07,6.339526e+06,-7.371000e+01,4.090000e+01,-7.371000e+01,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,...,2.650000e+02,5.000000e+00,3.984649e+07,3.609000e+03,3.080000e+03,4.226800e+04,9.117500e+04,3.000000e-01,3.984677e+07,2.750000e+00


In [4]:
initial.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'PUlon', 'PUlat', 'DOlon', 'DOlat',
       'PUmonth', 'PUhour', 'DOmonth', 'DOhour', 'PUweekday', 'DOweekday',
       'PUweekend', 'DOweekend', 'duration', 'passenger_count',
       'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge'],
      dtype='object')

In [20]:
#USE ONCE AT A TIME AND THEN LOCK IT BY COMMENTING SOMETHING like this(#)
initial[['fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'total_amount']]=initial[['fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'total_amount']]/100

In [9]:
initial

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,22012412,981079,-73.96,40.78,-73.96,40.77,11,11,11,11,...,141,1,7.0,0.0,0.5,2.06,0.0,0.3,12.36,2.5
1,9637084,5478693,-73.98,40.76,-73.95,40.78,2,20,2,21,...,263,1,10.0,0.5,0.5,2.76,0.0,0.3,16.55,2.5
2,1241948,2953331,-73.95,40.78,-73.99,40.75,1,7,1,7,...,164,1,13.5,0.0,0.5,3.36,0.0,0.3,20.16,2.5
3,4507018,1900930,-73.96,40.81,-73.94,40.82,1,0,1,0,...,42,1,5.5,0.5,0.5,0.00,0.0,0.3,6.80,0.0
4,16045351,8481,-73.95,40.77,-73.98,40.74,5,19,5,19,...,137,1,9.5,3.5,0.5,3.45,0.0,0.3,17.25,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383888,13897475,2604759,-73.98,40.74,-73.98,40.78,3,2,3,3,...,239,1,15.5,3.0,0.5,3.85,0.0,0.3,23.15,2.5
2383889,16333262,326818,-73.95,40.78,-73.96,40.78,6,19,6,20,...,236,1,4.0,1.0,0.5,1.00,0.0,0.3,9.30,2.5
2383890,5111796,1831234,-73.98,40.78,-73.98,40.78,1,19,1,19,...,239,2,4.5,3.5,0.5,0.00,0.0,0.3,8.80,2.5
2383891,9382670,1170205,-73.97,40.76,-73.98,40.74,2,16,2,16,...,107,1,5.0,1.0,0.5,1.86,0.0,0.3,11.16,2.5


In [21]:
dd= dict(zip(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],np.arange(0,7)))


In [22]:
initial['PUday']= initial['PUweekday'].map(dd)
initial['DOday']= initial['DOweekday'].map(dd)
initial['duration_sec']=initial['duration']*60

In [12]:
initial

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,PUday,DOday,duration_sec
0,22012412,981079,-73.96,40.78,-73.96,40.77,11,11,11,11,...,0.0,0.5,2.06,0.0,0.3,12.36,2.5,4,4,480
1,9637084,5478693,-73.98,40.76,-73.95,40.78,2,20,2,21,...,0.5,0.5,2.76,0.0,0.3,16.55,2.5,2,2,660
2,1241948,2953331,-73.95,40.78,-73.99,40.75,1,7,1,7,...,0.0,0.5,3.36,0.0,0.3,20.16,2.5,3,3,840
3,4507018,1900930,-73.96,40.81,-73.94,40.82,1,0,1,0,...,0.5,0.5,0.00,0.0,0.3,6.80,0.0,5,5,300
4,16045351,8481,-73.95,40.77,-73.98,40.74,5,19,5,19,...,3.5,0.5,3.45,0.0,0.3,17.25,2.5,4,4,540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383888,13897475,2604759,-73.98,40.74,-73.98,40.78,3,2,3,3,...,3.0,0.5,3.85,0.0,0.3,23.15,2.5,6,6,960
2383889,16333262,326818,-73.95,40.78,-73.96,40.78,6,19,6,20,...,1.0,0.5,1.00,0.0,0.3,9.30,2.5,0,0,120
2383890,5111796,1831234,-73.98,40.78,-73.98,40.78,1,19,1,19,...,3.5,0.5,0.00,0.0,0.3,8.80,2.5,4,4,180
2383891,9382670,1170205,-73.97,40.76,-73.98,40.74,2,16,2,16,...,1.0,0.5,1.86,0.0,0.3,11.16,2.5,3,3,240


In [23]:
initialdropped=initial.drop(['PUweekday','DOweekday','duration'],axis=1)

In [8]:
initialdropped

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,PUday,DOday,duration_sec
0,22012412,981079,-73.96,40.78,-73.96,40.77,11,11,11,11,...,0.0,0.5,2.06,0.0,0.3,12.36,2.5,4,4,480
1,9637084,5478693,-73.98,40.76,-73.95,40.78,2,20,2,21,...,0.5,0.5,2.76,0.0,0.3,16.55,2.5,2,2,660
2,1241948,2953331,-73.95,40.78,-73.99,40.75,1,7,1,7,...,0.0,0.5,3.36,0.0,0.3,20.16,2.5,3,3,840
3,4507018,1900930,-73.96,40.81,-73.94,40.82,1,0,1,0,...,0.5,0.5,0.00,0.0,0.3,6.80,0.0,5,5,300
4,16045351,8481,-73.95,40.77,-73.98,40.74,5,19,5,19,...,3.5,0.5,3.45,0.0,0.3,17.25,2.5,4,4,540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383888,13897475,2604759,-73.98,40.74,-73.98,40.78,3,2,3,3,...,3.0,0.5,3.85,0.0,0.3,23.15,2.5,6,6,960
2383889,16333262,326818,-73.95,40.78,-73.96,40.78,6,19,6,20,...,1.0,0.5,1.00,0.0,0.3,9.30,2.5,0,0,120
2383890,5111796,1831234,-73.98,40.78,-73.98,40.78,1,19,1,19,...,3.5,0.5,0.00,0.0,0.3,8.80,2.5,4,4,180
2383891,9382670,1170205,-73.97,40.76,-73.98,40.74,2,16,2,16,...,1.0,0.5,1.86,0.0,0.3,11.16,2.5,3,3,240


In [15]:
initialdropped.isnull().any()

Unnamed: 0               False
Unnamed: 0.1             False
PUlon                    False
PUlat                    False
DOlon                    False
DOlat                    False
PUmonth                  False
PUhour                   False
DOmonth                  False
DOhour                   False
PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
RatecodeID               False
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
PUday                    False
DOday                    False
duration_sec             False
dtype: bool

### HANDLING UNREALISTIC DATA AND CATEGORICAL DATA

In [24]:
idxp = initialdropped['payment_type']<=6
idxr=  initialdropped['RatecodeID']<=6

In [25]:
prfiltered= initialdropped[idxp & idxr]

In [26]:
pay= pd.get_dummies(prfiltered['payment_type'],drop_first=True)

In [27]:
payname= pay.rename(columns=dict(zip(np.arange(2,6),['pay2','pay3','pay4','pay5'])))

In [21]:
payname

,pay2,pay3,pay4,pay5
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
2383888,0,0,0,0
2383889,0,0,0,0
2383890,1,0,0,0
2383891,0,0,0,0


In [28]:
rate= pd.get_dummies(prfiltered['RatecodeID'],drop_first=True)

In [29]:
ratename = rate.rename(columns=dict(zip(np.arange(2,7),['r2','r3','r4','r5','r6'])))

In [24]:

ratename

,r2,r3,r4,r5,r6
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2383888,0,0,0,0,0
2383889,0,0,0,0,0
2383890,0,0,0,0,0
2383891,0,0,0,0,0


In [30]:
raw= pd.concat((prfiltered,ratename,payname),axis=1).drop('RatecodeID',axis=1) # keep payment_type column for task 3c

In [31]:
# LOCATION CONDITIONS
idxloc=raw['PULocationID']<=263
idx1loc=raw['DOLocationID']<=263

In [32]:
lofiltered= raw[idxloc & idx1loc]

In [28]:
lofiltered.describe()

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,...,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2357282.0
mean,1.191744e+07,2.050802e+06,-7.397149e+01,4.075532e+01,-7.397212e+01,4.075675e+01,4.385962e+00,1.390912e+01,4.386171e+00,1.398057e+01,...,8.867855e+02,1.779465e-02,1.540333e-03,9.926687e-05,3.196478e-03,4.242174e-06,2.572968e-01,5.776568e-03,3.425555e-03,0.0
std,6.881472e+06,1.820036e+06,3.814393e-02,2.874558e-02,3.365895e-02,3.181757e-02,3.829563e+00,5.549927e+00,3.829433e+00,5.616705e+00,...,3.692896e+03,1.322044e-01,3.921685e-02,9.962784e-03,5.644698e-02,2.059650e-03,4.371444e-01,7.578392e-02,5.842792e-02,0.0
min,1.000000e+00,1.000000e+00,-7.423000e+01,4.053000e+01,-7.423000e+01,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,5.953906e+06,5.514280e+05,-7.399000e+01,4.074000e+01,-7.399000e+01,4.074000e+01,1.000000e+00,1.000000e+01,1.000000e+00,1.000000e+01,...,3.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,1.191666e+07,1.331792e+06,-7.398000e+01,4.076000e+01,-7.398000e+01,4.076000e+01,2.000000e+00,1.400000e+01,2.000000e+00,1.500000e+01,...,6.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,1.788207e+07,3.313412e+06,-7.397000e+01,4.077000e+01,-7.396000e+01,4.078000e+01,8.000000e+00,1.800000e+01,8.000000e+00,1.800000e+01,...,9.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0
max,2.383893e+07,6.339526e+06,-7.371000e+01,4.090000e+01,-7.371000e+01,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,...,1.231800e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [33]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >=0
    #APPROACH 2 (Absolute values) 
    #### REPRODUCIBLE OR NOT COlumns when stating columns from 5 to the end???
lonilfiltered=np.abs(lofiltered.iloc[:,5::])
lonilfiltered0= pd.concat((lofiltered.iloc[:,0:5],lonilfiltered),axis=1)



In [34]:
lonilfiltered.describe()

,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,DOweekend,passenger_count,trip_distance,PULocationID,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,...,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2.357282e+06,2357282.0
mean,4.075675e+01,4.385962e+00,1.390912e+01,4.386171e+00,1.398057e+01,1.112565e-01,1.120999e-01,1.468970e+00,2.725240e+00,1.639951e+02,...,8.867855e+02,1.779465e-02,1.540333e-03,9.926687e-05,3.196478e-03,4.242174e-06,2.572968e-01,5.776568e-03,3.425555e-03,0.0
std,3.181757e-02,3.829563e+00,5.549927e+00,3.829433e+00,5.616705e+00,3.144496e-01,3.154893e-01,1.115593e+00,3.175411e+01,6.594467e+01,...,3.692896e+03,1.322044e-01,3.921685e-02,9.962784e-03,5.644698e-02,2.059650e-03,4.371444e-01,7.578392e-02,5.842792e-02,0.0
min,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,4.074000e+01,1.000000e+00,1.000000e+01,1.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00,9.700000e-01,1.250000e+02,...,3.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,4.076000e+01,2.000000e+00,1.400000e+01,2.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.600000e+00,1.620000e+02,...,6.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,4.078000e+01,8.000000e+00,1.800000e+01,8.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.870000e+00,2.340000e+02,...,9.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0
max,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,1.000000e+00,1.000000e+00,9.000000e+00,4.847745e+04,2.630000e+02,...,1.231800e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [35]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >0
    #APPROACH 1 
#DURATIOIN CONDITION
c01=  lonilfiltered0['duration_sec']>0
# PASSENGER COUNT CONDITION 
c02= lonilfiltered0['passenger_count']>0
#TRIP DISTANCE CONDITION
c03= lonilfiltered0['trip_distance']>0

lonilfiltered= lonilfiltered0[c01 & c02 &  c03]

In [36]:
# Speed conditions
sc1= (lonilfiltered['trip_distance']/lonilfiltered['duration_sec'])<=0.009 #17mil JFK to timesquare 45 min # < 34MPH 
sc2= (lonilfiltered['trip_distance']/lonilfiltered['duration_sec'])>0.00097 # >3.5MPH
lonilspfiltered= lonilfiltered[sc1 & sc2]

In [154]:
lonilspfiltered.describe()

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,...,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2.204373e+06,2204373.0
mean,1.189715e+07,2.059486e+06,-7.397376e+01,4.075644e+01,-7.397346e+01,4.075730e+01,4.378651e+00,1.394460e+01,4.378861e+00,1.402413e+01,...,7.165470e+02,1.155022e-02,8.401482e-04,6.033462e-05,8.052176e-04,2.268219e-06,2.547432e-01,3.981177e-03,2.778568e-03,0.0
std,6.889825e+06,1.821747e+06,3.298218e-02,2.642903e-02,2.994468e-02,3.016950e-02,3.835832e+00,5.509133e+00,3.835721e+00,5.571514e+00,...,5.648142e+02,1.068495e-01,2.897314e-02,7.767304e-03,2.836494e-02,1.506059e-03,4.357169e-01,6.297086e-02,5.263885e-02,0.0
min,1.000000e+00,1.000000e+00,-7.423000e+01,4.053000e+01,-7.423000e+01,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,5.930102e+06,5.590420e+05,-7.399000e+01,4.074000e+01,-7.399000e+01,4.074000e+01,1.000000e+00,1.000000e+01,1.000000e+00,1.000000e+01,...,3.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,1.185815e+07,1.339874e+06,-7.398000e+01,4.076000e+01,-7.398000e+01,4.076000e+01,2.000000e+00,1.400000e+01,2.000000e+00,1.500000e+01,...,6.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,1.790019e+07,3.327840e+06,-7.397000e+01,4.077000e+01,-7.396000e+01,4.078000e+01,8.000000e+00,1.800000e+01,8.000000e+00,1.800000e+01,...,9.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0
max,2.383893e+07,6.339490e+06,-7.371000e+01,4.090000e+01,-7.371000e+01,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,...,2.304000e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [37]:
# FARE AMOUNT CONDITIONS

fc1= lonilspfiltered['fare_amount']<=lonilspfiltered['trip_distance']*3+ ((lonilspfiltered['trip_distance']*0.5)/0.2)
fc2= lonilspfiltered['fare_amount']>lonilspfiltered['trip_distance']*2.5
lonilspffiltered= lonilspfiltered[fc1 & fc2]

In [22]:
lonilspffiltered

,Unnamed: 0,Unnamed: 0.1,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
1,9637084,5478693,-73.98,40.76,-73.95,40.78,2.0,20.0,2.0,21.0,...,660.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1241948,2953331,-73.95,40.78,-73.99,40.75,1.0,7.0,1.0,7.0,...,840.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16045351,8481,-73.95,40.77,-73.98,40.74,5.0,19.0,5.0,19.0,...,540.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,23760690,1294549,-73.95,40.79,-73.92,40.81,12.0,13.0,12.0,13.0,...,840.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,20850237,798146,-74.00,40.75,-73.97,40.77,10.0,12.0,10.0,12.0,...,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383882,12533848,978959,-73.97,40.76,-74.00,40.75,2.0,19.0,2.0,19.0,...,840.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2383886,20308313,898836,-74.00,40.75,-73.99,40.78,10.0,15.0,10.0,15.0,...,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2383887,2665649,5250474,-73.96,40.78,-73.94,40.80,1.0,7.0,1.0,8.0,...,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2383888,13897475,2604759,-73.98,40.74,-73.98,40.78,3.0,2.0,3.0,3.0,...,960.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
raw= lonilspffiltered

In [45]:
raw.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PUlon', 'PUlat', 'DOlon', 'DOlat',
       'PUmonth', 'PUhour', 'DOmonth', 'DOhour', 'PUweekend', 'DOweekend',
       'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'PUday', 'DOday', 'duration_sec', 'r2', 'r3',
       'r4', 'r5', 'r6', 'pay2', 'pay3', 'pay4', 'pay5'],
      dtype='object')

### AUGMENTED DATASET AND DOING THE TRANSOFMATION FOR PERIODICITY FEATURES

In [40]:
#SIN COS TRANSFORM FOR MONTH DAY HOUR
raw['PUhoursin']= np.sin(raw['PUhour']*(2.*np.pi/24)) # coordinate values on Oy
raw['PUhourcos']= np.cos(raw['PUhour']*(2.*np.pi/24)) #coordinate values on Ox
raw['DOhoursin']= np.sin(raw['DOhour']*(2.*np.pi/24)) # coordinate values on Oy
raw['DOhourcos']= np.cos(raw['DOhour']*(2.*np.pi/24)) #coordinate values on Ox

raw['PUdaysin']= np.sin(raw['PUday']*(2.*np.pi/7))
raw['PUdaycos']= np.cos(raw['PUday']*(2.*np.pi/7))
raw['DOdaysin']= np.sin(raw['DOday']*(2.*np.pi/7))
raw['DOdaycos']= np.cos(raw['DOday']*(2.*np.pi/7))


raw['PUmonthsin']= np.sin(raw['PUmonth']*(2.*np.pi/12))
raw['PUmonthcos']= np.cos(raw['PUmonth']*(2.*np.pi/12))
raw['DOmonthsin']= np.sin(raw['DOmonth']*(2.*np.pi/12))
raw['DOmonthcos']= np.cos(raw['DOmonth']*(2.*np.pi/12))


<ipython-input-40-5ef35a10511a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUhoursin']= np.sin(raw['PUhour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-40-5ef35a10511a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUhourcos']= np.cos(raw['PUhour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-40-5ef35a10511a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [41]:
raw.drop(['PUhour','DOhour','PUday','DOday','PUmonth','DOmonth'],axis=1,inplace=True)

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
raw.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PUlon', 'PUlat', 'DOlon', 'DOlat',
       'PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'duration_sec', 'r2', 'r3',
       'r4', 'r5', 'r6', 'pay2', 'pay3', 'pay4', 'pay5', 'PUhoursin',
       'PUhourcos', 'DOhoursin', 'DOhourcos', 'PUdaysin', 'PUdaycos',
       'DOdaysin', 'DOdaycos', 'PUmonthsin', 'PUmonthcos', 'DOmonthsin',
       'DOmonthcos'],
      dtype='object')

In [42]:
len(raw)

1206553

### HANDLING LONGITUDE LATITUDE FEATURES BASED ON LOCATIONID

In [43]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)
#JFKairport= (-73.78, 40.64)

#NWairport=(-74.172363, 40.735657)
NWairport=(-74.17, 40.74)

#Timesquare = (-73.98604331729206,40.75837054036599)
Timesquare = (-73.99,40.76)

#Wtc=  (-74.01335094122703,40.71315120718066)
Wtc=  (-74.01,40.71)

#Boa = (-73.9420433120473, 40.82320010657351)
Boa = (-73.94, 40.82)


L= [JFKairport, NWairport, Timesquare,Wtc,Boa]


In [44]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res


In [45]:
res= detora(L)

In [46]:
res 

[[-1.2877208169085756, 0.7093326305250857],
 [-1.2945107062041943, 0.7110471372624899],
 [-1.2913691135506042, 0.7113962031128886],
 [-1.2917181794010033, 0.7105235384868915],
 [-1.2904964489246071, 0.7124434006640853]]

In [47]:
# convert decimal lon lat to radians for PUlon  PUlat , DOlon, DOlat
raw['PUlonra']= np.radians(raw['PUlon'])
raw['PUlatra']= np.radians(raw['PUlat'])
raw['DOlonra']= np.radians(raw['DOlon'])
raw['DOlatra']= np.radians(raw['DOlat'])


<ipython-input-47-29ab1a99c552>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUlonra']= np.radians(raw['PUlon'])
<ipython-input-47-29ab1a99c552>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUlatra']= np.radians(raw['PUlat'])
<ipython-input-47-29ab1a99c552>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [48]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,len(res)):
        dlonp= raw['PUlonra']-res[i][0]
        dlatp= raw['PUlatra']-res[i][1]
        ap= np.sin(dlatp/2)**2 + np.cos(raw['PUlatra']) * np.cos(res[i][1]) * np.sin(dlonp/2)**2
        cp= 2*np.arcsin(np.sqrt(ap))
        sp=cp*3956
        sp=sp.rename(i)
        df= pd.concat((df,sp),axis=1)
    return df

In [49]:
#Compute the Harvesine distance between the PU DO 
def haversineindfpd(df):
    dlonpd= raw['PUlonra']-raw['DOlonra']
    dlatpd= raw['PUlatra']-raw['DOlatra']
    apd= np.sin(dlatpd/2)**2 + np.cos(raw['PUlatra']) * np.cos(raw['DOlatra']) * np.sin(dlonpd/2)**2
    cpd= 2*np.arcsin(np.sqrt(apd))
    spd=cpd*3956
    df= pd.concat((df,spd),axis=1)
    return df

In [50]:
s1= pd.DataFrame(np.zeros(len(raw.index)),index=raw.index)
predf= haversineindf(s1)
predfpd= haversineindfpd(s1)

In [36]:
predf

,0,0,1,2,3,4
1,0.0,13.235690,10.033664,0.522984,3.792304,4.640508
2,0.0,13.013770,11.832469,2.506347,5.762843,2.810831
4,0.0,12.516661,11.691458,2.202784,5.197541,3.491609
6,0.0,13.527863,12.011574,2.943592,6.352932,2.136273
10,0.0,13.686369,8.919493,0.866186,2.810934,5.761815
...,...,...,...,...,...,...
2383882,0.0,12.827728,10.551994,1.045968,4.037027,4.429612
2383886,0.0,13.686369,8.919493,0.866186,2.810934,5.761815
2383887,0.0,13.375060,11.324589,2.089920,5.495487,2.953019
2383888,0.0,12.432900,9.939679,1.476649,2.598984,5.906260


In [51]:
df=predf.iloc[:,1::]
dfpd=predfpd.iloc[:,1::]

In [59]:
df.describe()

,0,1,2,3,4
count,1.206553e+06,1.206553e+06,1.206553e+06,1.206553e+06,1.206553e+06
mean,1.274838e+01,1.064513e+01,2.165960e+00,4.104517e+00,5.156064e+00
std,2.269464e+00,2.288330e+00,2.299616e+00,2.353409e+00,2.316192e+00
min,7.396634e-01,3.452261e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.282199e+01,9.441111e+00,8.661856e-01,2.598984e+00,3.607077e+00
50%,1.323569e+01,1.003366e+01,1.476649e+00,3.792304e+00,4.898337e+00
75%,1.365153e+01,1.117709e+01,2.506347e+00,5.197541e+00,6.110977e+00
max,2.477737e+01,2.407256e+01,2.025523e+01,1.695274e+01,2.513036e+01


In [52]:
dfname=df.rename(columns=dict(zip(np.arange(0,10),['tojfk','tonw', 'totimes','towtc','toboa'])))
dfpdname=dfpd.rename(columns={0:'pd'})

In [53]:
Xfull= pd.concat((raw,dfname, dfpdname),axis=1)

In [54]:
Xfull.drop(['PUlon','PUlat','DOlon','DOlat', 'PUlonra','PUlatra','DOlonra','DOlatra','PULocationID','DOLocationID' ],axis=1,inplace=True)

In [55]:
Xfull

,Unnamed: 0,Unnamed: 0.1,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,...,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tonw,totimes,towtc,toboa,pd
1,9637084,5478693,0.0,0.0,1.0,2.42,1.0,10.0,0.5,0.5,...,8.660254e-01,5.000000e-01,8.660254e-01,5.000000e-01,13.235690,10.033664,0.522984,3.792304,4.640508,2.089920
2,1241948,2953331,0.0,0.0,1.0,3.38,1.0,13.5,0.0,0.5,...,5.000000e-01,8.660254e-01,5.000000e-01,8.660254e-01,13.013770,11.832469,2.506347,5.762843,2.810831,2.943816
4,16045351,8481,0.0,0.0,1.0,2.40,1.0,9.5,3.5,0.5,...,5.000000e-01,-8.660254e-01,5.000000e-01,-8.660254e-01,12.516661,11.691458,2.202784,5.197541,3.491609,2.598557
6,23760690,1294549,0.0,0.0,1.0,2.80,1.0,12.0,0.0,0.5,...,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000e+00,13.527863,12.011574,2.943592,6.352932,2.136273,2.089388
10,20850237,798146,0.0,0.0,2.0,2.60,1.0,10.5,2.5,0.5,...,-8.660254e-01,5.000000e-01,-8.660254e-01,5.000000e-01,13.686369,8.919493,0.866186,2.810934,5.761815,2.090097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383882,12533848,978959,0.0,0.0,1.0,2.37,1.0,11.0,1.0,0.5,...,8.660254e-01,5.000000e-01,8.660254e-01,5.000000e-01,12.827728,10.551994,1.045968,4.037027,4.429612,1.714265
2383886,20308313,898836,0.0,0.0,1.0,2.40,1.0,9.5,2.5,0.5,...,-8.660254e-01,5.000000e-01,-8.660254e-01,5.000000e-01,13.686369,8.919493,0.866186,2.810934,5.761815,2.136350
2383887,2665649,5250474,0.0,0.0,1.0,2.10,1.0,8.0,2.5,0.5,...,5.000000e-01,8.660254e-01,5.000000e-01,8.660254e-01,13.375060,11.324589,2.089920,5.495487,2.953019,1.732039
2383888,13897475,2604759,1.0,1.0,1.0,4.30,1.0,15.5,3.0,0.5,...,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17,12.432900,9.939679,1.476649,2.598984,5.906260,2.761809


In [64]:
Xfull.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PUweekend', 'DOweekend',
       'passenger_count', 'trip_distance', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6', 'pay2', 'pay3', 'pay4',
       'pay5', 'PUhoursin', 'PUhourcos', 'DOhoursin', 'DOhourcos', 'PUdaysin',
       'PUdaycos', 'DOdaysin', 'DOdaycos', 'PUmonthsin', 'PUmonthcos',
       'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw', 'totimes', 'towtc',
       'toboa', 'pd'],
      dtype='object')

### TRIMMING COLUMNS THAT WOULD NOT BE USED AT ALL FOR TASK 3 WHICH HAS NOT BEEN DEALT WITH FROM THE OUTSET
(NOte:Whenever I create a new column based on the available columns from the original table, I dropped those columns given that the newly created column would act as an useful alternative for the dropped columns already. However, there would be still  the Unnamed columns that were useful above still existing the table. Now, it is time to say goodbye to them)

In [65]:
Xfull.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PUweekend', 'DOweekend',
       'passenger_count', 'trip_distance', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6', 'pay2', 'pay3', 'pay4',
       'pay5', 'PUhoursin', 'PUhourcos', 'DOhoursin', 'DOhourcos', 'PUdaysin',
       'PUdaycos', 'DOdaysin', 'DOdaycos', 'PUmonthsin', 'PUmonthcos',
       'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw', 'totimes', 'towtc',
       'toboa', 'pd'],
      dtype='object')

In [56]:
Xfull['pay5'].unique() # since there is no variability of this feature in dataset

array([0.])

In [57]:
Xfull.drop(['Unnamed: 0','Unnamed: 0.1','pay5'],axis=1, inplace=True)

In [58]:
Xuse=Xfull.loc[:,~Xfull.columns.duplicated()]

In [42]:
Xuse.isnull().any()

PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration_sec             False
r2                       False
r3                       False
r4                       False
r5                       False
r6                       False
pay2                     False
pay3                     False
pay4                     False
PUhoursin                False
PUhourcos                False
DOhoursin                False
DOhourcos                False
PUdaysin                 False
PUdaycos                 False
DOdaysin                 False
DOdaycos                 False
PUmonthsin               False
PUmonthcos               False
DOmonths

### SCALING DATA

In [43]:
#why scaling: convergence and all features use the same scale to elimination the dominance of some features over others
#rs=RobustScaler()
#why robustscaler: outliers in trip_distance

In [155]:
#Xuse['trip_distance'].describe() 

count    1.206553e+06
mean     3.733359e+00
std      3.410459e+00
min      3.000000e-01
25%      1.750000e+00
50%      2.550000e+00
75%      4.200000e+00
max      1.626000e+02
Name: trip_distance, dtype: float64

In [44]:
#rs.fit(Xuse)

RobustScaler()

In [45]:
#Xscaled= pd.DataFrame(rs.transform(Xuse),index=Xuse.index,columns=Xuse.columns)


In [48]:
#Xscaled

,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tonw,totimes,towtc,toboa,pd
1,0.0,0.0,0.0,-0.053061,0.0,-0.1875,0.0,0.0,0.178571,0.0,...,2.679492e-01,0.000000e+00,2.679492e-01,0.000000e+00,0.000000,0.000000,-0.581446,0.000000,-0.102971,0.000000
2,0.0,0.0,0.0,0.338776,0.0,0.2500,-0.2,0.0,0.357143,0.0,...,0.000000e+00,4.226497e-01,0.000000e+00,4.226497e-01,-0.267521,1.036189,0.627803,0.758321,-0.833702,0.475559
4,0.0,0.0,0.0,-0.061224,0.0,-0.2500,1.2,0.0,0.383929,0.0,...,2.844589e-16,-1.577350e+00,2.844589e-16,-1.577350e+00,-0.866780,0.954961,0.442722,0.540776,-0.561815,0.283275
6,0.0,0.0,0.0,0.102041,0.0,0.0625,-0.2,0.0,0.113095,0.0,...,-3.660254e-01,5.773503e-01,-3.660254e-01,5.773503e-01,0.352211,1.139362,0.894390,0.985404,-1.103105,-0.000296
10,0.0,0.0,1.0,0.020408,0.0,-0.1250,0.8,0.0,0.175595,0.0,...,-1.000000e+00,-8.973826e-16,-1.000000e+00,-8.973826e-16,0.543288,-0.641810,-0.372197,-0.377660,0.344853,0.000099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383882,0.0,0.0,0.0,-0.073469,0.0,-0.0625,0.2,0.0,0.038690,0.0,...,2.679492e-01,0.000000e+00,2.679492e-01,0.000000e+00,-0.491793,0.298581,-0.262585,0.094177,-0.187198,-0.209213
2383886,0.0,0.0,0.0,-0.061224,0.0,-0.2500,0.8,0.0,0.113095,0.0,...,-1.000000e+00,-8.973826e-16,-1.000000e+00,-8.973826e-16,0.543288,-0.641810,-0.372197,-0.377660,0.344853,0.025858
2383887,0.0,0.0,0.0,-0.183673,0.0,-0.4375,0.8,0.0,-0.345238,0.0,...,0.000000e+00,4.226497e-01,0.000000e+00,4.226497e-01,0.168009,0.743629,0.373909,0.655434,-0.776915,-0.199314
2383888,1.0,1.0,0.0,0.714286,0.0,0.5000,1.0,0.0,0.502976,0.0,...,3.660254e-01,-5.773503e-01,3.660254e-01,-5.773503e-01,-0.967754,-0.054139,0.000000,-0.459224,0.402542,0.374194


### TASK 3

#### 3a Predicting trip distance

In [59]:
Xscaled= Xuse

In [75]:

Xscaled.columns

Index(['Unnamed: 0.1', 'PUweekend', 'DOweekend', 'passenger_count',
       'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw',
       'totimes', 'towtc', 'toboa', 'pd'],
      dtype='object')

In [60]:
ya=Xscaled['trip_distance']

In [61]:
Xa= Xscaled.drop(['trip_distance','pd','duration_sec','passenger_count','payment_type','pay2','pay3','pay4','r2','r3','r4','r5','r6',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'DOweekend','DOhoursin', 'DOhourcos','DOdaysin','DOdaycos','DOmonthsin','DOmonthcos'],axis=1)


In [62]:
Xa.columns

Index(['PUweekend', 'PUhoursin', 'PUhourcos', 'PUdaysin', 'PUdaycos',
       'PUmonthsin', 'PUmonthcos', 'tojfk', 'tonw', 'totimes', 'towtc',
       'toboa'],
      dtype='object')

In [63]:
Xa

,PUweekend,PUhoursin,PUhourcos,PUdaysin,PUdaycos,PUmonthsin,PUmonthcos,tojfk,tonw,totimes,towtc,toboa
1,0.0,-8.660254e-01,0.500000,0.974928,-0.222521,8.660254e-01,5.000000e-01,13.235690,10.033664,0.522984,3.792304,4.640508
2,0.0,9.659258e-01,-0.258819,0.433884,-0.900969,5.000000e-01,8.660254e-01,13.013770,11.832469,2.506347,5.762843,2.810831
4,0.0,-9.659258e-01,0.258819,-0.433884,-0.900969,5.000000e-01,-8.660254e-01,12.516661,11.691458,2.202784,5.197541,3.491609
6,0.0,-2.588190e-01,-0.965926,0.974928,-0.222521,-2.449294e-16,1.000000e+00,13.527863,12.011574,2.943592,6.352932,2.136273
10,0.0,1.224647e-16,-1.000000,-0.974928,-0.222521,-8.660254e-01,5.000000e-01,13.686369,8.919493,0.866186,2.810934,5.761815
...,...,...,...,...,...,...,...,...,...,...,...,...
2383882,0.0,-9.659258e-01,0.258819,0.974928,-0.222521,8.660254e-01,5.000000e-01,12.827728,10.551994,1.045968,4.037027,4.429612
2383886,0.0,-7.071068e-01,-0.707107,0.000000,1.000000,-8.660254e-01,5.000000e-01,13.686369,8.919493,0.866186,2.810934,5.761815
2383887,0.0,9.659258e-01,-0.258819,0.000000,1.000000,5.000000e-01,8.660254e-01,13.375060,11.324589,2.089920,5.495487,2.953019
2383888,1.0,5.000000e-01,0.866025,-0.781831,0.623490,1.000000e+00,6.123234e-17,12.432900,9.939679,1.476649,2.598984,5.906260


In [53]:
Xa.isnull().any()

PUweekend     False
PUhoursin     False
PUhourcos     False
PUdaysin      False
PUdaycos      False
PUmonthsin    False
PUmonthcos    False
tojfk         False
tonw          False
totimes       False
towtc         False
toboa         False
dtype: bool

In [64]:
Xtrain,Xtest,ytrain,ytest= train_test_split(Xa,ya,test_size=0.2, random_state= 0)

In [65]:
ytraindf= pd.DataFrame(pd.Series(ytrain,index=ytrain.index),index=ytrain.index)
ytestdf= pd.DataFrame(pd.Series(ytest,index=ytest.index),index=ytest.index)


In [66]:
transformer_x = RobustScaler().fit(Xtrain)
transformer_y = RobustScaler().fit(ytraindf)
X_train = pd.DataFrame(transformer_x.transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
y_train = transformer_y.transform(ytraindf)
X_test = pd.DataFrame(transformer_x.transform(Xtest), index=Xtest.index, columns=Xtest.columns)
y_test = transformer_y.transform(ytestdf)

In [67]:
X_train

,PUweekend,PUhoursin,PUhourcos,PUdaysin,PUdaycos,PUmonthsin,PUmonthcos,tojfk,tonw,totimes,towtc,toboa
2377518,0.0,-0.414214,4.288254e-01,0.623490,0.000000e+00,-1.000000,-8.973826e-16,0.104857,-0.656842,2.875466e-13,-0.637230,0.580653
49839,0.0,1.214413,-1.998008e-01,-0.277479,-4.450419e-01,-0.732051,-1.577350e+00,-0.498706,-0.040771,-3.721971e-01,-0.236902,0.146898
1942627,1.0,-0.325401,-1.998008e-01,-0.500000,5.549581e-01,0.000000,4.226497e-01,0.543288,-0.641810,-3.721971e-01,-0.377660,0.344853
1775417,0.0,1.303225,7.357911e-16,-0.277479,-4.450419e-01,0.267949,0.000000e+00,1.203842,0.564669,5.144079e-01,0.813641,-0.918764
422402,0.0,0.214413,-5.857864e-01,-0.623490,-1.638616e-16,-0.366025,-1.732051e+00,0.018323,-0.341336,-4.793411e-01,-0.322831,0.238189
...,...,...,...,...,...,...,...,...,...,...,...,...
711432,1.0,-0.414214,0.000000e+00,-0.500000,5.549581e-01,-0.732051,-1.577350e+00,-0.498706,-0.040771,-3.721971e-01,-0.236902,0.146898
300798,0.0,1.303225,7.357911e-16,0.000000,8.019377e-01,0.366025,-5.773503e-01,-0.414380,0.658664,1.447417e-01,0.426038,-0.515699
1904043,0.0,1.303225,7.357911e-16,0.623490,0.000000e+00,0.000000,4.226497e-01,-1.006974,0.259869,-1.361321e-01,-0.125872,0.073057
232477,0.0,-0.414214,0.000000e+00,0.000000,8.019377e-01,0.267949,0.000000e+00,0.000000,0.000000,-5.814460e-01,0.000000,-0.102971


In [68]:
# Which model, which hyperparameters to use? I do Cross-validation to check
modelhya= SGDRegressor(random_state=0)
hyperparameters= {'loss':['squared_loss','huber','epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 3)} 
#,'squared_epsilon_insensitive'

In [69]:
grid= GridSearchCV(modelhya, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [70]:
grid.fit(X_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(cv=5, estimator=SGDRegressor(random_state=0), n_jobs=-1,
             param_grid={'alpha': array([0.    , 0.0005, 0.001 ]),
                         'loss': ['squared_loss', 'huber',
                                  'epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [71]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','ep0','squared0.0005','huber0.0005','ep.0005','squared0.001','huber0.001','ep0.001'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [72]:
comparison #=> alpha =0, loss= squared is the best( see the results on real queens dataset). USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.129548
huber0,1.182688
ep0,1.183951
squared0.0005,1.129574
huber0.0005,1.181238
ep.0005,1.184113
squared0.001,1.129628
huber0.001,1.180303
ep0.001,1.183647


In [73]:
modela =SGDRegressor(alpha= 0, random_state=0)

In [74]:
modela.fit(X_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SGDRegressor(alpha=0, random_state=0)

In [75]:
modela.score(X_train,y_train)

0.34431762258278376

In [76]:
modela.score(X_test,y_test)

0.35271512915322256

In [80]:
transformer_y.inverse_transform(modela.predict(X_test).reshape(len(X_test),1))

array([[ 3.53037478],
       [13.65037676],
       [ 4.46975192],
       ...,
       [ 3.37161826],
       [ 3.26100367],
       [ 3.51097561]])

#### 3b Predicting fare_amount (parsimonious model)

In [81]:
yb= Xscaled['fare_amount']

In [82]:
Xscaled.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw',
       'totimes', 'towtc', 'toboa', 'pd'],
      dtype='object')

In [84]:
Xb =Xscaled.drop(['fare_amount','payment_type',
                      'extra',
                     'mta_tax',            'tip_amount',
                'tolls_amount', 'improvement_surcharge',
                'total_amount',  'congestion_surcharge','trip_distance','duration_sec'
                ],axis=1)

In [85]:
Xb.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'r2', 'r3', 'r4', 'r5',
       'r6', 'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw',
       'totimes', 'towtc', 'toboa', 'pd'],
      dtype='object')

In [122]:
Xb

,PUweekend,DOweekend,passenger_count,r2,r3,r4,r5,r6,pay2,pay3,...,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tonw,totimes,towtc,toboa,pd
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.679492e-01,0.000000e+00,2.679492e-01,0.000000e+00,0.000000,0.000000,-0.581446,0.000000,-0.102971,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,4.226497e-01,0.000000e+00,4.226497e-01,-0.267521,1.036189,0.627803,0.758321,-0.833702,0.475559
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.844589e-16,-1.577350e+00,2.844589e-16,-1.577350e+00,-0.866780,0.954961,0.442722,0.540776,-0.561815,0.283275
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.660254e-01,5.773503e-01,-3.660254e-01,5.773503e-01,0.352211,1.139362,0.894390,0.985404,-1.103105,-0.000296
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000e+00,-8.973826e-16,-1.000000e+00,-8.973826e-16,0.543288,-0.641810,-0.372197,-0.377660,0.344853,0.000099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.679492e-01,0.000000e+00,2.679492e-01,0.000000e+00,-0.491793,0.298581,-0.262585,0.094177,-0.187198,-0.209213
2383886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000e+00,-8.973826e-16,-1.000000e+00,-8.973826e-16,0.543288,-0.641810,-0.372197,-0.377660,0.344853,0.025858
2383887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,4.226497e-01,0.000000e+00,4.226497e-01,0.168009,0.743629,0.373909,0.655434,-0.776915,-0.199314
2383888,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.660254e-01,-5.773503e-01,3.660254e-01,-5.773503e-01,-0.967754,-0.054139,0.000000,-0.459224,0.402542,0.374194


In [86]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xb,yb,test_size=0.2, random_state= 0)

In [87]:
ytraindf= pd.DataFrame(pd.Series(ytrain,index=ytrain.index),index=ytrain.index)
ytestdf= pd.DataFrame(pd.Series(ytest,index=ytest.index),index=ytest.index)

In [88]:
transformer_x = RobustScaler().fit(Xtrain)
transformer_y = RobustScaler().fit(ytraindf)
X_train = pd.DataFrame(transformer_x.transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
y_train = transformer_y.transform(ytraindf)
X_test = pd.DataFrame(transformer_x.transform(Xtest), index=Xtest.index, columns=Xtest.columns)
y_test = transformer_y.transform(ytestdf)

In [89]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyb= SGDRegressor(random_state=0)
hyperparameters= {'loss':['squared_loss','huber'], 'alpha': [0,0.0001]} 


In [90]:
grid= GridSearchCV(modelhyb, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [91]:
grid.fit(X_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(cv=5, estimator=SGDRegressor(random_state=0), n_jobs=-1,
             param_grid={'alpha': [0, 0.0001],
                         'loss': ['squared_loss', 'huber']},
             scoring='neg_root_mean_squared_error')

In [92]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared0.0001','huber0.0001']) 
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [93]:
comparison #=> alpha = 0, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,0.471150
huber0,0.489688
squared0.0001,0.471351
huber0.0001,0.489985


In [94]:
# doing cross validation to find out how many features to go for
from sklearn.model_selection import cross_val_score
modelb = SGDRegressor(alpha=0,random_state=0)
scores = cross_val_score(modelb, X_train, y_train.flatten(), cv=5)


In [95]:
np.mean(scores)

0.855487635212055

In [136]:
Xscaled.corr()


,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tonw,totimes,towtc,toboa,pd
PUweekend,1.000000,0.988901,0.017716,-0.014425,0.009551,-0.032201,-0.062237,0.004042,-0.027178,-0.012522,...,0.040833,0.015923,0.040781,0.016006,-0.003580,-0.009286,0.004056,-0.017660,0.019276,-0.014838
DOweekend,0.988901,1.000000,0.017993,-0.013853,0.008832,-0.031273,-0.061517,0.004130,-0.026536,-0.012962,...,0.041775,0.016536,0.041861,0.016205,-0.002809,-0.011103,0.003037,-0.019897,0.020007,-0.014215
passenger_count,0.017716,0.017993,1.000000,0.007555,0.006100,0.008240,-0.131969,-0.000577,-0.001548,0.006465,...,0.029707,0.027723,0.029714,0.027685,-0.003402,-0.003464,0.003795,-0.007056,0.011762,0.002973
trip_distance,-0.014425,-0.013853,0.007555,1.000000,0.027682,0.978286,0.015461,-0.145428,0.476698,0.581319,...,0.085484,0.037938,0.085509,0.037906,-0.549884,0.504431,0.544220,0.367409,0.408899,0.937732
payment_type,0.009551,0.008832,0.006100,0.027682,1.000000,0.023384,-0.024061,0.001823,-0.507546,-0.021304,...,-0.024196,-0.051248,-0.024199,-0.051228,-0.067605,0.094320,0.086327,0.091546,0.016564,0.024017
fare_amount,-0.032201,-0.031273,0.008240,0.978286,0.023384,1.000000,0.018484,-0.200700,0.487529,0.572566,...,0.092697,0.051005,0.092727,0.050957,-0.500891,0.458274,0.498985,0.330328,0.379004,0.914771
extra,-0.062237,-0.061517,-0.131969,0.015461,-0.024061,0.018484,1.000000,0.018538,0.036287,0.014729,...,0.024302,-0.012343,0.024310,-0.012354,0.020482,-0.066606,-0.059708,-0.075309,0.024200,0.032824
mta_tax,0.004042,0.004130,-0.000577,-0.145428,0.001823,-0.200700,0.018538,1.000000,-0.118208,-0.318014,...,-0.011910,-0.002957,-0.011903,-0.002966,0.013663,-0.005637,-0.009201,-0.000072,-0.017699,-0.126115
tip_amount,-0.027178,-0.026536,-0.001548,0.476698,-0.507546,0.487529,0.036287,-0.118208,1.000000,0.383587,...,0.071124,0.057981,0.071129,0.057955,-0.210597,0.177143,0.194067,0.112379,0.169998,0.442188
tolls_amount,-0.012522,-0.012962,0.006465,0.581319,-0.021304,0.572566,0.014729,-0.318014,0.383587,1.000000,...,0.077693,0.040900,0.077673,0.040925,-0.280047,0.317916,0.291069,0.250411,0.151523,0.523809


##### 7 strongest features: pd + 5 distance features + r2

In [96]:
modelb7 = SGDRegressor(alpha=0,random_state=0)
scoresb7 = cross_val_score(modelb7, X_train[['pd','tojfk',
       'tonw', 'totimes', 'towtc','toboa','r2' ]], y_train.flatten(), cv=5)

In [97]:
np.mean(scoresb7)

0.8400374536987052

##### Could the model be more parsimonious? 1 feature only. PU DO distance

In [98]:
modelb1= SGDRegressor(alpha=0,random_state=0)
scoresb1 = cross_val_score(modelb1, pd.DataFrame(X_train['pd'],index=X_train.index,columns=['pd']), y_train.flatten(), cv=5)

In [99]:
np.mean(scoresb1)

0.8380767189926068

####### => conclusion: 1 feature should be best both in terms of the performance of the model and the number of parameters. Let's see how the 1 feature perform on the test set

In [100]:
finalmodelb= SGDRegressor(alpha=0,random_state=0)
finalmodelb.fit(X_train[['pd']],y_train.flatten())

SGDRegressor(alpha=0, random_state=0)

In [101]:
finalmodelb.score(X_train[['pd']],y_train)

0.8381319882285158

In [102]:
finalmodelb.score(X_test[['pd']],y_test)

0.8313601566161476

In [106]:
transformer_y.inverse_transform(finalmodelb.predict(X_test[['pd']]).reshape(len(X_test['pd']),1))

array([[ 9.60696784],
       [46.35903621],
       [11.95867719],
       ...,
       [17.47256881],
       [10.61900847],
       [ 9.60674746]])

#### 3c Predict payment type

In [107]:
Xscaled.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk', 'tonw',
       'totimes', 'towtc', 'toboa', 'pd'],
      dtype='object')

In [108]:
yc= Xscaled['payment_type']

In [109]:
Xc=Xscaled.drop(['payment_type', 'pay2', 'pay3', 'pay4'],axis=1)

In [110]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xc,yc,test_size=0.2, random_state=0)

In [111]:
ytraindf= pd.DataFrame(pd.Series(ytrain,index=ytrain.index),index=ytrain.index)
ytestdf= pd.DataFrame(pd.Series(ytest,index=ytest.index),index=ytest.index)

In [112]:
transformer_x = RobustScaler().fit(Xtrain)
transformer_y = RobustScaler().fit(ytraindf)
X_train = pd.DataFrame(transformer_x.transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
y_train = transformer_y.transform(ytraindf)
X_test = pd.DataFrame(transformer_x.transform(Xtest), index=Xtest.index, columns=Xtest.columns)
y_test = transformer_y.transform(ytestdf)

In [113]:
X_train.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6', 'PUhoursin', 'PUhourcos',
       'DOhoursin', 'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin',
       'DOdaycos', 'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos',
       'tojfk', 'tonw', 'totimes', 'towtc', 'toboa', 'pd'],
      dtype='object')

In [149]:
X_train.isnull().any()

PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration_sec             False
r2                       False
r3                       False
r4                       False
r5                       False
r6                       False
PUhoursin                False
PUhourcos                False
DOhoursin                False
DOhourcos                False
PUdaysin                 False
PUdaycos                 False
DOdaysin                 False
DOdaycos                 False
PUmonthsin               False
PUmonthcos               False
DOmonthsin               False
DOmonthcos               False
tojfk                    False
tonw                     False
totimes 

In [114]:
modelclog= SGDClassifier(loss='log',random_state=0)
scoreclog = cross_val_score(modelclog,X_train,y_train.flatten(), cv=5)




In [115]:
np.mean(scoreclog)

0.968005951945653

In [116]:
modelcsvc= SGDClassifier(loss='hinge',random_state=0)
scorecsvc = cross_val_score(modelcsvc, X_train, y_train.flatten(), cv=5)

In [117]:
np.mean(scorecsvc)

0.968123020878434

=> use svc

In [118]:
modelc= SGDClassifier(loss='hinge',random_state=0)


In [119]:
modelc.fit(X_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SGDClassifier(random_state=0)

In [120]:
modelc.score(X_train,y_train)


0.9681219839169867

In [121]:
modelc.score(X_test,y_test)

0.9681406981032775

In [122]:
transformer_y.inverse_transform(modelc.predict(X_test).reshape(len(X_test),1))

array([[2.],
       [1.],
       [2.],
       ...,
       [1.],
       [1.],
       [1.]])